**Table of contents**<a id='toc0_'></a>    
- [Time-series | Theory](#toc1_)    
  - [Time-series types](#toc1_1_)    
    - [From a statistical perspective](#toc1_1_1_)    
      - [Examples of non-stationary series](#toc1_1_1_1_)    
      - [**Why is stationarity important?**](#toc1_1_1_2_)    
      - [White noise](#toc1_1_1_3_)    
      - [Random Walk](#toc1_1_1_4_)    
      - [Random Walk Differencing](#toc1_1_1_5_)    
    - [From a prediction perspective](#toc1_1_2_)    
  - [Time-series models](#toc1_2_)    
  - [Time series elements](#toc1_3_)    
    - [Trend](#toc1_3_1_)    
    - [Seasonality](#toc1_3_2_)    
    - [Cyclicity](#toc1_3_3_)    
- [Time-series | Practice](#toc2_)    
  - [EDA | $CO_2$ levels in a room](#toc2_1_)    
    - [Data Wrangling](#toc2_1_1_)    
    - [Time series decomposition](#toc2_1_2_)    
    - [Autocorrelation](#toc2_1_3_)    
      - [Manual check](#toc2_1_3_1_)    
      - [Autocorrelation plot](#toc2_1_3_2_)    
      - [Statistical test (Augmented Dickey-Fuller)](#toc2_1_3_3_)    
  - [Curve Smoothing | Stock Prices](#toc2_2_)    
    - [Moving Average Smoothing](#toc2_2_1_)    
    - [Exponential Smoothing](#toc2_2_2_)    
  - [Classical Model | Air passenger prediction](#toc2_3_)    
    - [ARIMA family of models](#toc2_3_1_)    
      - [ARMA (Autoregressive Moving Average Model)](#toc2_3_1_1_)    
      - [ARIMA model (Autoregressive Integrated Moving Average Model)](#toc2_3_1_2_)    
      - [SARIMA model (Seasonal Autoregressive Moving Average Model)](#toc2_3_1_3_)    
      - [SARIMAX model (Seasonal Autoregressive Moving Average Model)](#toc2_3_1_4_)    
  - [Machine Learning Model | Stock Price Prediction](#toc2_4_)    
- [References](#toc3_)    
- [Extra](#toc4_)    
- [Acknowledgements](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
# You know the drill
# !pip install statsmodels
# !pip install -q yfinance
# !pip install pmdarima --quiet
# !pip install tensorflow
# !pip install keras

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
import yfinance as yf
import pmdarima as pm
import warnings

warnings.filterwarnings("ignore")
pio.templates.default = "simple_white"
px.defaults.template = "ggplot2"
px.defaults.color_continuous_scale = px.colors.sequential.Blackbody

# <a id='toc1_'></a>[Time-series | Theory](#toc0_)

![](https://media.giphy.com/media/NTMxntb8rMzmbd1x97/giphy.gif)

> Time series forecasting is exactly what it sounds like; predicting unknown values. Time series forecasting involves the collection of historical data, preparing it for algorithms to consume, and then predicting the future values based on patterns learned from the historical data.
> There are numerous reasons why companies may be interested in forecasting future values, namely GDP, monthly sales, inventory, unemployment, and global temperatures:
> - A retailer may be interested in <span style="color:orange">predicting future sales</span> at an SKU (stock keeping unit) level for planning and budgeting.
> - A small merchant may be interested in <span style="color:orange">forecasting sales</span> by store, so it can schedule the right resources (more people during busy periods and vice versa).
> - A software giant like Google may be interested in <span style="color:orange">knowing the busiest hour of the day or busiest day of the week</span> so that they can schedule server resources accordingly.
> - The health department may be interested in <span style="color:orange">predicting the cumulative COVID vaccinations administered</span> so that they can further predict when herd immunity is expected to kick in. [$^{[1]}$](https://www.datacamp.com/tutorial/tutorial-time-series-forecasting)

In short, time series forecasting allows us to **predict numerical continuous or discrete values in the future given historical data**.

## <a id='toc1_1_'></a>[Time-series types](#toc0_)

### <a id='toc1_1_1_'></a>[From a statistical perspective](#toc0_)

- **Stationary** - the time series doesn't change its statistical properties (mean and variance) and the points are not autocorrelated.
- **Non-stationary** - the time series changes any of its statistical properties (mean and variance) and/or the points are autocorrelated.

#### <a id='toc1_1_1_1_'></a>[Examples of non-stationary series](#toc0_)

**1. Mean changes over time**   
Green: Stationary series, Red: Non-stationary series  

![](https://habrastorage.org/files/20c/9d8/a63/20c9d8a633ec436f91dccd4aedcc6940.png)    
(Source: [Sean Abu](https://www.seanabu.com/2016/03/22/time-series-seasonal-ARIMA-model-in-python/))

**2. Variance changes over time**   
Green: Stationary series, Red: Non-stationary series  

![](https://habrastorage.org/files/b88/eec/a67/b88eeca676d642449cab135273fd5a95.png)  
(Source: [Sean Abu](https://www.seanabu.com/2016/03/22/time-series-seasonal-ARIMA-model-in-python/))


**3. Autocorrelation is present**   
Green: Stationary series, Red: Non-stationary series  
> - Finally, the covariance of the i th term and the (i + m) th term should not be a function of time. In the following graph, you will notice that the spread becomes closer as time increases. Hence, the covariance is not constant with time in the right chart. $^{[2]}$

![](https://habrastorage.org/files/2f6/1ee/cb2/2f61eecb20714352840748b826e38680.png)    
(Source: [Sean Abu](https://www.seanabu.com/2016/03/22/time-series-seasonal-ARIMA-model-in-python/))

#### <a id='toc1_1_1_2_'></a>[**Why is stationarity important?**](#toc0_)

TL;DR: It makes predictions easier.

> It is easy to make predictions on a stationary series since we can assume that the future statistical properties will not be different from those currently observed. Most of the time-series models, in one way or the other, try to predict those properties (mean or variance, for example). Future predictions would be wrong if the original series were not stationary. Unfortunately, most of the time series that we see outside of textbooks are non-stationary, but we can (and should) change this.

> So, in order to combat non-stationarity, we have to know our enemy, so to speak. White noise and random walks are useful to understanding how to convert a non-stationary time series into a stationary time series.

#### <a id='toc1_1_1_3_'></a>[White noise](#toc0_)

> A time series is white noise if the **variables are independent** and **identically distributed** with a **mean of zero**.
> 
> It is important in time series & forecasting for two main reasons:
> - **Predictability**: If your time series is white noise, then, by definition, it is random. You cannot reasonably model it and make predictions.
> - **Model Diagnostics**: The series of errors from a time series forecast model should ideally be white noise.[$^{[5]}$](https://machinelearningmastery.com/white-noise-time-series-python/)

In [ ]:
noise = np.random.normal(loc=0, scale=1, size=5000)
plt.plot(noise)
plt.show()

In [ ]:
plot_acf(noise, lags=50, alpha=0.05)
plt.show()

#### <a id='toc1_1_1_4_'></a>[Random Walk](#toc0_)

A random walk is simply white noise with a starting point, i.e. points are autocorrelated.

![](https://upload.wikimedia.org/wikipedia/commons/2/28/Eight-step_random_walks.png)   
(Source: [Wikipedia](https://en.wikipedia.org/wiki/Random_walk))

> Random walks are an incredibly useful model for stochastic processes across many applications, such as modeling the **movement of particles through a fluid**, **the search path of a foraging animal**, or **changes in stock prices**. [$^{[6]}$](https://archive.ph/20211114104840/https://towardsdatascience.com/a-deep-dive-on-arima-models-8900c199ccf)

In [ ]:
# draw samples from a standard Normal model distribution (mean=0, std=1)
points = np.random.standard_normal(1000) # kicks random walk (how it chooses to jump)

# simulate the random walk
points[0]=0
random_walk = np.cumsum(points)
plt.figure(figsize=(10, 7.5))
plt.plot(random_walk)
plt.title("simulated random walk")
plt.show()

In [ ]:
plot_acf(random_walk, lags=50, alpha=0.05)
plt.show()

In [ ]:
plot_pacf(random_walk, lags=50, alpha=0.05)
plt.show()

Is the random walk stationary?

#### <a id='toc1_1_1_5_'></a>[Random Walk Differencing](#toc0_)

In time series analysis, many times we aim to transform non-stationary series to stationary series. To do so, one of the common methods used is **differencing**:

In [ ]:
random_walk_df = pd.DataFrame(random_walk, columns=['yesterday'])
random_walk_df['today'] = random_walk_df.shift(-1)
random_walk_df['difference'] = random_walk_df['today'] - random_walk_df['yesterday']
random_walk_df

In [ ]:
random_walk_diff = random_walk[1:] - random_walk[:-1]
plt.figure(figsize=(10, 7.5))
plt.plot(random_walk_diff)
plt.title("simulated random walk 1st order difference")
plt.show()

In [ ]:
plot_acf(random_walk_diff, lags=50, alpha=0.05)
plt.show()

### <a id='toc1_1_2_'></a>[From a prediction perspective](#toc0_)

- **Univariate** - prediction based on one time-dependent variable (i.e. we extract features from the target data)
- **Multivariate** - prediction based on multiple time-dependent variable (i.e. we extract features from the target data and/or other time-dependent variables)

In **multivariate time series forecasting**, the input variables can be of two types:
- **Exogenous**: Input variables are independent from other input variables and they influence the output variable.
- **Endogenous**: Input variables are influenced by other input variables (dependent) and they influence the output variable.

This means there are 3 types of multivariate time series models: fully exogenous, fully endogenous, and mixed.

## <a id='toc1_2_'></a>[Time-series models](#toc0_)

> Time series forecasting can broadly be categorized into the following categories:
> - **Classical / Statistical Models** — Moving Averages, Exponential Smoothing, ARIMA, SARIMA, TBATS
> - **Machine Learning** — Linear Regression, XGBoost, Random Forest, or any ML model with reduction methods
> - **Deep Learning** — RNN, LSTM [$^{[1]}$](https://www.datacamp.com/tutorial/tutorial-time-series-forecasting)

## <a id='toc1_3_'></a>[Time series elements](#toc0_)

### <a id='toc1_3_1_'></a>[Trend](#toc0_)

> A trend exists when there is a **long-term increase or decrease** in the data. It does not have to be linear. Sometimes we will refer to a trend as “changing direction”, when it might go from an increasing trend to a decreasing trend. [$^{[3]}$](https://otexts.com/fpp2/tspatterns.html)

### <a id='toc1_3_2_'></a>[Seasonality](#toc0_)

> A seasonal pattern occurs when a time series is affected by seasonal factors such as the time of the year or the day of the week. **Seasonality is always of a fixed and known frequency**.[$^{[3]}$](https://otexts.com/fpp2/tspatterns.html)

### <a id='toc1_3_3_'></a>[Cyclicity](#toc0_)

> A cycle occurs **when the data exhibit rises and falls that are not of a fixed frequency**. These fluctuations are usually due to economic conditions, and are often related to the “business cycle”. The duration of these fluctuations is usually at least 2 years. [$^{[3]}$](https://otexts.com/fpp2/tspatterns.html)

![](https://otexts.com/fpp2/fpp_files/figure-html/fourexamples-1.png)  
(Source: [OTexts](https://otexts.com/fpp2/tspatterns.html))

# <a id='toc2_'></a>[Time-series | Practice](#toc0_)

## <a id='toc2_1_'></a>[EDA | $CO_2$ levels in a room](#toc0_)

Today we'll be looking at a room occupancy dataset, which contains meta-data about the room (temperature, humidity, light, $CO_2$) and whether there was a person in the room or not (occupancy):

In [ ]:
occupancy = pd.read_csv('https://raw.githubusercontent.com/sabinagio/data-analytics/main/data/occupancy.csv')
occupancy.head()

### <a id='toc2_1_1_'></a>[Data Wrangling](#toc0_)

As we'll explore time series, we will correctly format the dates and use them as the index for our dataframe:

In [ ]:
occupancy['date'] = pd.to_datetime(occupancy['date'])
occupancy.index = pd.DatetimeIndex(occupancy['date'], freq='H')
occupancy.drop('date', axis=1, inplace=True) # do not run this twice
display(occupancy.head())

Now let's review what we can model:

In [ ]:
plotly_df = occupancy.stack().reset_index().rename({'level_1': 'metric', 0: 'value'}, axis=1)
fig = px.line(plotly_df, x='date', y='value', facet_row='metric')
# Have the y-axis adjust for each plot independently
fig.update_yaxes(matches=None, title='')

# Remove pattern=pattern from the y-axis label
fig.for_each_annotation(lambda annot: annot.update(text=annot.text.split("=")[-1]))

# Hide tick marks
fig.update_xaxes(ticks='')

# Follow previous aesthetics
fig.update_traces(line_color='black')
fig.update_layout(height=600, width=700)
fig.show()

### <a id='toc2_1_2_'></a>[Time series decomposition](#toc0_)

We can extract the trend and seasonality of our time series using the `statsmodel` library:

In [ ]:
decomp = sm.tsa.seasonal_decompose(occupancy['CO2'][:1000])
decomp.plot()
plt.show()

While `matplotlib` is decent for a basic chart, I'd like to display my data using `plotly` instead. However, for that I need to do some data wrangling with my time series:

In [ ]:
decomp_df = pd.concat([decomp.trend, decomp.seasonal, decomp.resid], axis=1)
decomp_df.head(20)

To show 3 lines in the same figure or chart using `plotly.express`, my data needs to have the features (trend, seasonality, and residuals) in a separate column:

In [ ]:
decomp_df = decomp_df.stack().reset_index(name='value').rename({'level_1': 'pattern'}, axis=1)
decomp_df

In [ ]:
fig = px.line(decomp_df, x='date', y='value', facet_row='pattern', title='CO2 level')

# Have the y-axis adjust for each plot independently
fig.update_yaxes(matches=None, title='')

# Remove pattern=pattern from the y-axis label
fig.for_each_annotation(lambda annot: annot.update(text=annot.text.split("=")[-1]))

# Hide tick marks
fig.update_xaxes(ticks='')

# Follow previous aesthetics
fig.update_traces(line_color='black')
fig.update_layout(height=600, width=700)
fig.show()

### <a id='toc2_1_3_'></a>[Autocorrelation](#toc0_)

> Autocorrelation measures the relationship between a variable's current value and its past values. [$^{[4]}$](https://www.investopedia.com/terms/a/autocorrelation.asp)

#### <a id='toc2_1_3_1_'></a>[Manual check](#toc0_)

In [ ]:
# Present hour values
occupancy['CO2'][:-1]

In [ ]:
# Past hour values
occupancy['CO2'][1:]

In [ ]:
fig = px.scatter(x=occupancy['CO2'][1:], y=occupancy['CO2'][:-1], title='CO2 levels autocorrelation')
fig.update_xaxes(title='Past hour')
fig.update_yaxes(title='Present hour')
fig.update_traces(opacity=0.5)
fig.show()

#### <a id='toc2_1_3_2_'></a>[Autocorrelation plot](#toc0_)

Simple autocorrelation:

In [ ]:
plot_acf(occupancy['CO2'], alpha=0.05)
plt.show()

Partial autocorrelation:

In [ ]:
plot_pacf(occupancy['CO2'], alpha=0.05)
plt.show()

After differencing the CO2 levels:

In [ ]:
# Extract the first order difference between CO2 levels
lvl_changes = ((occupancy['CO2'] - occupancy['CO2'].shift(1))/occupancy['CO2'].shift(1)).dropna()
lvl_changes

In [ ]:
plot_acf(lvl_changes, alpha=0.05)
plt.show()

In [ ]:
plot_pacf(lvl_changes, alpha=0.05)
plt.show()

#### <a id='toc2_1_3_3_'></a>[Statistical test (Augmented Dickey-Fuller)](#toc0_)

- Null Hypothesis $H_0$ = Time series is not stationary
- Alternative Hypothesis $H_1$ = Time series is stationary

In [ ]:
# function source: https://archive.ph/20220324144523/https://towardsdatascience.com/why-does-stationarity-matter-in-time-series-analysis-e2fb7be74454#selection-919.361-937.115
def ADF_Cal(x):
    result = adfuller(x, autolag='AIC')
    ADF_stat = result[0]
    p = result[1]
    print("ADF Statistic: %f" % ADF_stat)
    print("p-value: %f" % p)
    print("Critical Values")
    levels = [.01, .05, .1]
    i = 0
    for key,value in result[4].items():
        print('\t%s: %.3f' % (key,value))
        hyp = p < levels[i]
        if ADF_stat < value:
            cert = (1-levels[i])*100
            print("{}% certain this is stationary".format(cert))
            print('Reject H0: {}'.format(hyp))
            break
        i = i+1
        if i >= 3:
            print("Less than 90% certain that data is stationary")
            print('Reject H0: {}'.format(hyp))

In [ ]:
print("Calculating ADF test for X...")
ADF_Cal(occupancy['CO2'])

In [ ]:
print("Calculating ADF test for X...")
ADF_Cal(lvl_changes)

## <a id='toc2_2_'></a>[Curve Smoothing | Stock Prices](#toc0_)

In [ ]:
# Get data from Yahoo Finance
apple_df = yf.download('AAPL', start='2012-06-01', end='2022-06-30')
apple_df.head()

In [ ]:
# If the above doesn't work
# apple_df = pd.read_csv("https://raw.githubusercontent.com/sabinagio/data-analytics/main/data/AAPL.csv")
# apple_df.head()

In [ ]:
apple_df["open_mean"]= apple_df["Open"].mean()

fig = px.line(x= apple_df.index, y= apple_df["Open"])
fig.add_trace(px.line(x= apple_df.index, y= apple_df["open_mean"]).data[0])
fig.show()

### <a id='toc2_2_1_'></a>[Moving Average Smoothing](#toc0_)

In [ ]:
y = apple_df["Open"]
MA7 = y.rolling(window=7)
apple_df["Open_MA_7"]= MA7.mean()

In [ ]:
fig = px.line(x=apple_df.index, y= apple_df.Open)
fig.add_trace(px.line(x=apple_df.index, y= apple_df["Open_MA_7"]).data[0])
fig["data"][1]["line"]["color"]= "#4BE8E0"
fig.show()

### <a id='toc2_2_2_'></a>[Exponential Smoothing](#toc0_)

MA = sum(data_points_in_a_window) / window_size

Y(t) = alpha * Y(t) + (1-alpha) * Y(t-1)  
Y(t-1) = alpha * Y(t-1) + (1-alpha) * Y(t-2)  

Y(t) =  alpha * Y(t) + (1-alpha) * alpha * Y(t-1) +  (1-alpha) * (1 - alpha) * Y(t-2)

Y(t) = 0.1 * Y(t) + 0.09 * Y(t-1) + 0.81 * Y(t-1)

In [ ]:
SES = SimpleExpSmoothing(apple_df["Open"])
SES_fit = SES.fit(0.1)
SES_predict = SES_fit.predict(start=0, end=len(apple_df) + 30)

fig = px.line(x=range(len(apple_df)), y= apple_df["Open"])
fig.add_trace(px.line(x=SES_predict.index, y= SES_predict).data[0])
fig["data"][1]["line"]["color"]= "#4BE8E0"
fig.show()

## <a id='toc2_3_'></a>[Classical Model | Air passenger prediction](#toc0_)

In [ ]:
passengers = pd.read_csv("https://raw.githubusercontent.com/sabinagio/data-analytics/main/data/AirPassengers.csv")
passengers

In [ ]:
passengers['Month'] = pd.to_datetime(passengers['Month'])
passengers.index = pd.DatetimeIndex(passengers['Month'], freq='MS')
passengers.drop('Month', axis=1, inplace=True) # do not run this twice
display(passengers.head())

In [ ]:
fig = px.line(x=passengers.index, y= passengers["#Passengers"])
fig.show()

In [ ]:
decomp = sm.tsa.seasonal_decompose(passengers['#Passengers'][:1000])
decomp.plot()
plt.show()

### <a id='toc2_3_1_'></a>[ARIMA family of models](#toc0_)

#### <a id='toc2_3_1_1_'></a>[ARMA (Autoregressive Moving Average Model)](#toc0_)

> **ARMA = AR(p) + MA(q)**
> * **AR(p)**: autoregressive component that actually assumes that the current state of the series depends on its previous values with some lag p (the maximum period, or how far back a model looks to figure out the prediction) $^{[2]}$

![](https://miro.medium.com/max/1400/1*8If_nqKxNbzd3JPZAXICVg.png) 
(Source: [Medium](https://miro.medium.com/max/1400/1*8If_nqKxNbzd3JPZAXICVg.png))  

> * **MA(q)**: moving average component with q lagged forecasting error terms in the prediction. $^{[2]}$

#### <a id='toc2_3_1_2_'></a>[ARIMA model (Autoregressive Integrated Moving Average Model)](#toc0_)

> **ARIMA = AR(p) + I(d) + MA(q)**
> * **I(d)**: to model the number of non-seasonal differences needed to make the series stationary $^{[2]}$

In [ ]:
# standard ARIMA model
ARIMA_model = pm.auto_arima(passengers["#Passengers"],
                           start_p=1,
                           start_q=1,
                           test="adf",
                           max_p=3, max_q=3,
                           m=1, # frequency of the series if m==1 seasonal is set to FALSE automatically
                           d=None, # parameter that the algo will look for
                           seasonal=False, # no seasonality for std arima
                           trace=False, #logs
                           error_action="warn",
                           suppress_warnings=True,
                           stepwise=True)

print(ARIMA_model.summary())

In [ ]:
# diagnostics
ARIMA_model.plot_diagnostics(figsize=(15,12))
plt.show()

In [ ]:
def forecast(ARIMA_model, n_periods=24):
    fitted, confint = ARIMA_model.predict(n_periods=n_periods, return_conf_int=True)
    index_of_fc = pd.date_range(passengers.index[-1]+pd.DateOffset(months=1), periods=n_periods, freq="MS")

    # make the series for the plot
    fitted_series = pd.Series(fitted, index=index_of_fc)
    lower_series = pd.Series(confint[:,0], index=index_of_fc)
    upper_series = pd.Series(confint[:,1], index=index_of_fc)


    #plot
    plt.figure(figsize=(15, 7))
    plt.plot(passengers["#Passengers"], color="#1f76b4")
    plt.plot(fitted_series, color="darkgreen")
    plt.fill_between(lower_series.index,
                    lower_series,
                    upper_series,
                    color = "k",
                     alpha =.15)
    plt.title("ARIMA-Forecast Air Passengers")
    plt.show()

In [ ]:
forecast(ARIMA_model, n_periods=24)

#### <a id='toc2_3_1_3_'></a>[SARIMA model (Seasonal Autoregressive Moving Average Model)](#toc0_)
> **SARIMA = S(s) + AR(p) + MA(q) + I(d)**
> * **S(s)**: responsable for the seasonality and equals the season period length of the series $^{[2]}$

In [ ]:
# Seasonal - fit stepwise auto-ARIMA
SARIMA_model = pm.auto_arima(passengers['#Passengers'], start_p=1, start_q=1,
                         test="adf",
                         max_p=3, max_q=3,
                         m=12, #12 is the frequncy of the cycle
                         start_P=0,
                         seasonal=True, #set to seasonal
                         d=None,
                         D=1, #order of the seasonal differencing
                         trace=False,
                         error_action="ignore",
                         suppress_warnings=True,
                         stepwise=True)

In [ ]:
SARIMA_model.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
forecast(SARIMA_model)

#### <a id='toc2_3_1_4_'></a>[SARIMAX model (Seasonal Autoregressive Moving Average Model with Exogenous Factors)](#toc0_)
> **SARIMAX = S(s) + AR(p) + MA(q) + I(d) + X(t)**
> * **X**: eXogenous variable $^{[2]}$

In [ ]:
#adding exogenous variable
passengers['month_index'] = passengers.index.month
passengers

In [ ]:
# SARIMAX Model
SARIMAX_model = pm.auto_arima(passengers[['#Passengers']], exogenous=passengers[['month_index']],
                           start_p=1, start_q=1,
                           test='adf',
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=None, D=1,
                           trace=False,
                           error_action='ignore',
                           suppress_warnings=True,
                           stepwise=True)

SARIMAX_model.summary()

We can see that similar to the SARIMA model we are getting some pretty good-looking predictions and the width of the forecasted confidence interval has decreased. This means that the model is more certain of its predictions.

In [ ]:
def sarimax_forecast(SARIMAX_model, periods=24):
    # Forecast
    n_periods = periods
    forecast_passengers = pd.DataFrame({"month_index":pd.date_range(passengers.index[-1], periods = n_periods, freq="MS").month},
                    index = pd.date_range(passengers.index[-1]+ pd.DateOffset(months=1), periods = n_periods, freq="MS"))
    fitted, confint = SARIMAX_model.predict(n_periods=n_periods,
                                            return_conf_int=True,
                                            exogenous=forecast_passengers[["month_index"]])
    index_of_fc = pd.date_range(passengers.index[-1] + pd.DateOffset(months=1), periods = n_periods, freq="MS")
    # make series for plotting purpose
    fitted_series = pd.Series(fitted, index=index_of_fc)
    lower_series = pd.Series(confint[:, 0], index=index_of_fc)
    upper_series = pd.Series(confint[:, 1], index=index_of_fc)

    # Plot
    plt.figure(figsize=(15,7))
    plt.plot(passengers["#Passengers"], color="#1F76B4")
    plt.plot(fitted_series, color="darkgreen")
    plt.fill_between(lower_series.index,
                    lower_series,
                    upper_series,
                    color="k", alpha=.15)
    plt.title("SARIMAX - Forecast of Airline Passengers")
    plt.show()

In [ ]:
sarimax_forecast(SARIMAX_model, periods=24)

## <a id='toc2_4_'></a>[Deep Learning Model | Stock Price Prediction](#toc0_)

Following code adapted from [Analytics Vidhya](https://www.analyticsvidhya.com/blog/2021/12/stock-price-prediction-using-lstm/):

In [ ]:
apple_df_pred = apple_df[['Open', 'Close']]
apple_df_pred.head()

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
apple_df_scaled = scaler.fit_transform(apple_df_pred)

In [ ]:
training_size = round(len(apple_df_scaled) * 0.80)
train_data = apple_df_scaled[:training_size]
test_data  = apple_df_scaled[training_size:]

In [ ]:
def create_sequence(dataset):
  sequences = []
  labels = []

  start_idx = 0

  for stop_idx in range(50, len(dataset)): 
    sequences.append(dataset[start_idx:stop_idx])
    labels.append(dataset[stop_idx])
    start_idx += 1
  return (np.array(sequences),np.array(labels))

In [ ]:
train_seq, train_label = create_sequence(train_data)
test_seq, test_label = create_sequence(test_data)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape = (train_seq.shape[1], train_seq.shape[2])))
model.add(Dropout(0.1)) 
model.add(LSTM(units=50))
model.add(Dense(2))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()

In [ ]:
model.fit(train_seq, train_label, epochs=20, validation_data=(test_seq, test_label), verbose=1)
test_predicted = model.predict(test_seq)

In [ ]:
test_inverse_predicted = scaler.inverse_transform(test_predicted)

In [ ]:
# Merging actual and predicted data for better visualization
apple_pred_data = pd.concat([apple_df.iloc[-457:].copy(),pd.DataFrame(test_inverse_predicted,columns=['open_predicted','close_predicted'],index=apple_df .iloc[-457:].index)], axis=1)
apple_pred_data.head()

In [ ]:
open_plot_data = apple_pred_data[['Open', 'open_predicted']].stack().reset_index().rename({'level_1': 'Measure', 0: 'Value'}, axis=1)
close_plot_data = apple_pred_data[['Close', 'close_predicted']].stack().reset_index().rename({'level_1': 'Measure', 0: 'Value'}, axis=1)

In [ ]:
px.line(open_plot_data, x='Date', color='Measure', y='Value')

In [ ]:
px.line(close_plot_data, x='Date', color='Measure', y='Value')

# <a id='toc3_'></a>[References](#toc0_)

Unless otherwise cited, the quoted paragraphs belong to David Henriques.

[1] [Time Series Forecasting, DataCamp](https://www.datacamp.com/tutorial/tutorial-time-series-forecasting)  
[2] Time Series Crash Course (Ironhack July 2022 RMT PT bootcamp resources), Andre Oliveira Gomez    
[3] [Time series patterns, OTexts](https://otexts.com/fpp2/tspatterns.html)  
[4] [Autocorrelation, Investopedia](https://www.investopedia.com/terms/a/autocorrelation.asp)  
[5] [White Noise, Machine Learning Mastery](https://machinelearningmastery.com/white-noise-time-series-python/)  
[6] [A Deep Dive on ARIMA models, Towards Data Science](https://archive.ph/20211114104840/https://towardsdatascience.com/a-deep-dive-on-arima-models-8900c199ccf)

# <a id='toc4_'></a>[Extra](#toc0_)

Want to go deeper into time series models?
- [ARIMA & SARIMA Real World Time Series Forecasting, Neptune AI](https://neptune.ai/blog/arima-sarima-real-world-time-series-forecasting-guide)

If you want to learn about deep learning time series models:
- [RNNs - StatQuest (17 mins)](https://www.youtube.com/watch?v=AsNTP8Kwu80)
- [LSTMs - StatQuest (21 mins)](https://www.youtube.com/watch?v=YCzL96nL7j0)

If you're interested in algorithmic trading:
- [PyQuant newsletter](https://pyquantnews.com/the-pyquant-newsletter/)

# <a id='toc5_'></a>[Acknowledgements](#toc0_)

Thank you, Andre de Oliveira Gomes for your awesome lessons and resources on time series.